In [1]:
import h5py
import numpy as np
from scipy import linalg

/Users/atsushi/.pyenv/versions/anaconda3-5.1.0/envs/jupyter-env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# 直交表の読み出し
* https://hondou.homedns.org/pukiwiki/index.php?Python%20HDF5 で、直交表を　numpy.ndarray 形式で HDF5 ファイルにまとめた。
* それを読み出して使う
* 直交表について
$$
L_n(q^m) = OA (n, m, q, t)
$$
    * n : 大きさ、テスト回数
    * m : 因子数 = テストデータの項目数
    * q : 水準数 = テスト項目の値の種類
    * t : 強さ = テスト項目の組み合わせ数

In [2]:
    hdf5 = h5py.File("mypkg/oamatrix.hdf5", 'r')
    m=2
    q=3
    oa = hdf5["{}^{}".format(m,q)].value
    print(oa)
    hdf5.close()

[[0 0 0]
 [0 1 1]
 [1 0 1]
 [1 1 0]]


# 直交表と最小自乗推定で、パラメータの寄与度を計算する

In [3]:
# 直交表の最終列に、{1,1,1,1}T を追加する。　⇒ 誤差というか、嵩上げされている得点の部分
# いったん転置して、最終行に {1,1,1,1} を付け足してから転置してもとに戻す
m = oa.T
fixed = np.array([np.ones(oa.shape[0])]);
print(m)
print (fixed)

                 
A = (np.concatenate( (m, fixed) )).T
B = np.array([10.0,70.0,80.0,90.0])
print ("A=\n{}".format(A))
print ("B={}".format(B))
# least squares
C, resid, rank, sigma = linalg.lstsq(A, B)
print ("C={}".format(C))
print ("resid={}".format(resid))
print ("rank={}".format(rank))
print ("sigma={}".format(sigma))


[[0 0 1 1]
 [0 1 0 1]
 [0 1 1 0]]
[[1. 1. 1. 1.]]
A=
[[0. 0. 0. 1.]
 [0. 1. 1. 1.]
 [1. 0. 1. 1.]
 [1. 1. 0. 1.]]
B=[10. 70. 80. 90.]
C=[45. 35. 25. 10.]
resid=[]
rank=4
sigma=[2.73205081 1.         1.         0.73205081]


# 実験計画に使える直交表

In [4]:
def createOA (hdf5, n, m, q) :
    name = "{}^{}".format(m,q)
    if name in list(hdf5.keys()) :
        oa = hdf5[name].value
        print("L{}({}^{})".format(n,m,q))
        print(oa)
    else :
        print("L{}({}^{}) does not exist on my hdf5.".format(n,m,q))

hdf5 = h5py.File("mypkg/oamatrix.hdf5", 'r')

createOA(hdf5, 4,2,3)
createOA(hdf5, 12,2,11)
createOA(hdf5, 20,2,19)
createOA(hdf5, 24,2,23)
createOA(hdf5, 44,2,43)
createOA(hdf5, 60,2,59)
createOA(hdf5, 9,3,4)
createOA(hdf5, 27,3,13)
createOA(hdf5, 16,4,5)
createOA(hdf5, 64,4,21)
createOA(hdf5, 25,5,6)
createOA(hdf5, 50,5,12)

hdf5.close()


L4(2^3)
[[0 0 0]
 [0 1 1]
 [1 0 1]
 [1 1 0]]
L12(2^11)
[[0 0 0 1 0 0 1 0 1 1 1]
 [0 0 1 0 0 1 0 1 1 1 0]
 [0 0 1 0 1 1 1 0 0 0 1]
 [0 1 0 0 1 0 1 1 1 0 0]
 [0 1 0 1 1 1 0 0 0 1 0]
 [0 1 1 1 0 0 0 1 0 0 1]
 [1 0 0 0 1 0 0 1 0 1 1]
 [1 0 0 1 0 1 1 1 0 0 0]
 [1 0 1 1 1 0 0 0 1 0 0]
 [1 1 0 0 0 1 0 0 1 0 1]
 [1 1 1 0 0 0 1 0 0 1 0]
 [1 1 1 1 1 1 1 1 1 1 1]]
L20(2^19)
[[0 0 0 0 1 1 0 0 1 0 0 1 1 1 1 0 1 0 1]
 [0 0 0 1 1 0 0 1 0 0 1 1 1 1 0 1 0 1 0]
 [0 0 1 0 0 1 1 1 1 0 1 0 1 0 0 0 0 1 1]
 [0 0 1 1 0 0 1 0 0 1 1 1 1 0 1 0 1 0 0]
 [0 0 1 1 1 1 0 1 0 1 0 0 0 0 1 1 0 0 1]
 [0 1 0 0 0 0 1 1 0 0 1 0 0 1 1 1 1 0 1]
 [0 1 0 0 1 1 1 1 0 1 0 1 0 0 0 0 1 1 0]
 [0 1 0 1 0 0 0 0 1 1 0 0 1 0 0 1 1 1 1]
 [0 1 1 0 0 1 0 0 1 1 1 1 0 1 0 1 0 0 0]
 [0 1 1 1 1 0 1 0 1 0 0 0 0 1 1 0 0 1 0]
 [1 0 0 0 0 1 1 0 0 1 0 0 1 1 1 1 0 1 0]
 [1 0 0 1 0 0 1 1 1 1 0 1 0 1 0 0 0 0 1]
 [1 0 0 1 1 1 1 0 1 0 1 0 0 0 0 1 1 0 0]
 [1 0 1 0 0 0 0 1 1 0 0 1 0 0 1 1 1 1 0]
 [1 0 1 0 1 0 0 0 0 1 1 0 0 1 0 0 1 1 1]
 [1 1 0 0 1 0 0 1 1